In [1]:
#ProstateX final 

#Andrii Trokoz



'''
Importing all the necessary libraries and frameworks in order to start image processing 
'''
import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import os
import fnmatch
import shutil
from ipywidgets import interact, fixed
import pandas as pd
import warnings


def myshow(image_arr,slice_n):
    plt.imshow(image_arr[slice_n,:,:],cmap='gray')
    plt.show()

In [2]:
#reading .csv in order to get all of necessary coordinates for cropping 
data_image = pd.read_csv('C://Image Processing/ProstateX-TestLesionInformation/ProstateX-TrainingLesionInformationv2/ProstateX-Images-Train.csv')



In [3]:
#Extracting the names of each image for the further processing from Data Frame, we need this to match names of files with pos coordinates
data_image.loc[:,'match'] = '' #creates new column where we`ll put the matching names 

for i in range(len(data_image)):  #names for further matching with images` names
    data_image.loc[i,'match'] = data_image.loc[i,'DCMSerDescr'] + '_' + data_image.loc[i,'ProxID'].replace('-','_') + '.nii.gz'



In [5]:
'''
Creating separate DataFrames for each of modalites. 

It is going to be: t2_df, adc_df, bval_df and ktrans_df

'''



#creating lists for further processing, renaming and removing the duplicates in the directory  
t2_dir=os.listdir('C://Image Processing/ProstateX/resampled/')
t2_match = [i for i in t2_dir if 't2' in i ]

def make_match_list(directory, matching_value): 
    #generating a list of matching names
    match=[i for i in directory if matching_value in i]
    for i in range(len(match)): 
        if match[i].split('_')[-1][0] !='0':
            match[i]=match[i][:-12] + '.nii.gz'
    
    match_list=[]
    for i in match: 
        if i not in match_list:
            match_list.append(i)
            
    return match_list

#macth lists
t2_list = make_match_list(t2_dir,'t2')

adc_list = make_match_list(t2_dir,'ADC')

bval_list = make_match_list(t2_dir, 'BVAL')



#matching dataframes

#Creating a DataFrame with names of files 

t2_df = data_image[data_image['match'].isin(t2_list)].copy() #use .copy() to overcome SettingWithCopyWarning issue
t2_df.index = range(len(t2_df))

adc_df = data_image[data_image['match'].isin(adc_list)].copy()
adc_df.index = range(len(adc_df))

bval_df = data_image[data_image['match'].isin(bval_list)].copy()
bval_df.index = range(len(bval_df))

In [6]:
#to match all of the files with pos coordinates, we have to assign names to all of our 'match' names
#here we are defining a function to assign names to instances that have different DCMSerNum, but the same name 
#these names are the same as in our files` directory
#function design specifically for this project, cause names of columns in the dataframes and indexing are only suitable for 
#ProstateX data
def rename_missed_match(data):
   #data=corresponding dataframe
    try:
        for i in range(len(data)):
        
            if data.loc[i,'match']==data.loc[i+1,'match'] and data.loc[i,'DCMSerNum'] != data.loc[i+1,'DCMSerNum']:
            
                if len(str(data.loc[i,'DCMSerNum'])) == 1:
                    if len(str(data.loc[i+1,'DCMSerNum'])) != 2:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
                    else: 
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
            
                elif len(str(data.loc[i,'DCMSerNum'])) == 2:
                    if len(str(data.loc[i+1,'DCMSerNum'])) != 2:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i+1,'match'].strip('.nii.gz')+'_s00'+str(data.loc[i+1,'DCMSerNum'])+'.nii.gz'
                    else:
                        data.loc[i,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
                        data.loc[i+1,'match'] = data.loc[i,'match'].strip('.nii.gz')+'_s0'+str(data.loc[i,'DCMSerNum'])+'.nii.gz'
    except: KeyError


rename_missed_match(t2_df)  #applying functions
rename_missed_match(adc_df)
rename_missed_match(bval_df)




In [ ]:
#Making new column with physical coordinates of the findings 
#In order to do that, we should first read image, than get pos and finally transfer physical point to index

#first of all we should match ProxID and read image 

#for reading image we have to undestand which image to read, get its path and match it with correct pos

#find ProxID -- find filename -- match proxid with filename -- find pos -- read image -- transform image pos to index

def phys_to_index_col(data, folder):
    directory = os.listdir(folder)
    for i in directory: 
        for k in data['ProxID']:
            if k.split('-')[-1] in i: 
                pos=[float(i) for i in (data['pos'].loc[data['ProxID']==k][0]).split(' ')]
                im = sitk.ReadImage(folder+i)
                data['index'].loc[data['ProxID']==k]=im.TransformPhysicalPointToIndex(pos)



In [85]:
#DataFrame.set_value(index, col, value, takeable=False)
#df.at[4, 'B']
a = (1,2,3)
b = (10,20,30)
t2_df.at[1,'coordinates'] = b
t2_df


,ProxID,Name,fid,pos,WorldMatrix,ijk,TopLevel,SpacingBetweenSlices,VoxelSpacing,Dim,DCMSerDescr,DCMSerNum,match,coordinates
0,ProstateX-0000,t2_tse_tra0,1,25.7457 31.8707 -38.511,"0.5,1.00168e-010,0.00377059,-57.9373,-0.000199...",167 224 9,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0000.nii.gz,"(1, 2, 3)"
1,ProstateX-0001,t2_tse_tra0,1,-40.5367071921656 29.320722668457 -16.70766907...,"0.5,1.02552e-010,-4.64581e-017,-119.205,-1.019...",157 186 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,6,t2_tse_tra_ProstateX_0001_s006.nii.gz,"(10, 20, 30)"
2,ProstateX-0001,t2_tse_tra0,1,-40.5367071921656 29.320722668457 -16.70766907...,"0.5,1.02552e-010,-4.64581e-017,-119.205,-1.019...",157 186 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,10,t2_tse_tra_ProstateX_0001_s010.nii.gz,"(1, 2, 3)"
3,ProstateX-0002,t2_tse_tra0,2,-2.058 38.6752 -34.6104,"0.5,1.02552e-010,7.98512e-017,-103.784,-9.9634...",203 216 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0002.nii.gz,"(1, 2, 3)"
4,ProstateX-0002,t2_tse_tra0,1,-27.0102 41.5467 -26.0469,"0.5,1.02552e-010,7.98512e-017,-103.784,-9.9634...",154 217 12,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0002.nii.gz,"(1, 2, 3)"
5,ProstateX-0003,t2_tse_tra0,1,22.1495 31.2717 -2.45933,"0.5,1.02552e-010,3.21757e-017,-98.0005,-8.5615...",240 205 12,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,3,t2_tse_tra_ProstateX_0003.nii.gz,"(1, 2, 3)"
6,ProstateX-0003,t2_tse_tra0,2,-21.2871 19.3995 19.7429,"0.5,1.02552e-010,3.21757e-017,-98.0005,-8.5615...",153 161 17,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,3,t2_tse_tra_ProstateX_0003.nii.gz,"(1, 2, 3)"
7,ProstateX-0004,t2_tse_tra0,1,-7.69665 3.64226 23.1659,"0.5,1.02552e-010,6.33633e-017,-118.163,-9.9505...",221 168 7,1.0,4.5,"0.5,0.5,4.5",384x384x19x1,t2_tse_tra,5,t2_tse_tra_ProstateX_0004.nii.gz,"(1, 2, 3)"
8,ProstateX-0005,t2_tse_tra0,1,-22.0892639160156 25.4668045043945 22.87915420...,"0.5,1.02552e-010,1.10673e-016,-117.278,-1.0216...",190 185 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0005.nii.gz,"(1, 2, 3)"
9,ProstateX-0005,t2_tse_tra0,0,-14.5174331665039 49.4428329467773 20.78152465...,"0.5,1.02552e-010,1.10673e-016,-117.278,-1.0216...",206 233 10,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0005.nii.gz,"(1, 2, 3)"


In [87]:
t2_df.loc[:,'coordinates'] = ''
#t2_dir=os.listdir('C://Image Processing/ProstateX/resampled/')
folder='C://Image Processing/ProstateX/resampled/'
for i in range(len(t2_df)):
    for image in t2_dir:
        if t2_df.loc[i,'match'] ==image:
            try:
                #print(t2_df.loc[i,'pos'])
                #print(i)
                pos=[float(i) for i in t2_df.loc[i,'pos'].split(' ')]
            except: 
                if ValueError:
                    print(t2_df.loc[i,'pos'].split(' '), '\nEERRRRRRRRORRRRRRR ERRRRRRRORRRRRRR ERRRRRRROOOORRRRRRRRRRR')
                    t2_df.loc[i, 'pos'] = t2_df.loc[i, 'pos'].strip(' ')
                    pos=[float(i) for i in t2_df.loc[i,'pos'].split(' ')]
                    print(t2_df.loc[i, 'pos'].split(' '), '\tERROR solved')
            im=sitk.ReadImage(folder+image)
            ind=im.TransformPhysicalPointToIndex(pos)
            t2_df.at[i,'coordinates']=ind
            
            print(pos, '\tare coordinates for\t'+t2_df.loc[i,'match']+'\ttransformed to\t'+str(t2_df.loc[i,'coordinates']))
            

[25.7457, 31.8707, -38.511] 	are coordinates for	t2_tse_tra_ProstateX_0000.nii.gz	transformed to	(167, 159, 8)
[-40.5367071921656, 29.320722668457, -16.7076690704346] 	are coordinates for	t2_tse_tra_ProstateX_0001_s006.nii.gz	transformed to	(157, 197, 9)
[-40.5367071921656, 29.320722668457, -16.7076690704346] 	are coordinates for	t2_tse_tra_ProstateX_0001_s010.nii.gz	transformed to	(157, 197, 9)
[-2.058, 38.6752, -34.6104] 	are coordinates for	t2_tse_tra_ProstateX_0002.nii.gz	transformed to	(203, 168, 9)
[-27.0102, 41.5467, -26.0469] 	are coordinates for	t2_tse_tra_ProstateX_0002.nii.gz	transformed to	(153, 166, 12)
[22.1495, 31.2717, -2.45933] 	are coordinates for	t2_tse_tra_ProstateX_0003.nii.gz	transformed to	(240, 178, 12)
[-21.2871, 19.3995, 19.7429] 	are coordinates for	t2_tse_tra_ProstateX_0003.nii.gz	transformed to	(153, 222, 16)
[-7.69665, 3.64226, 23.1659] 	are coordinates for	t2_tse_tra_ProstateX_0004.nii.gz	transformed to	(220, 216, 10)
[-22.0892639160156, 25.4668045043945,

[-11.24582, 60.9975, -48.0648] 	are coordinates for	t2_tse_tra_ProstateX_0041.nii.gz	transformed to	(150, 149, 6)
[0.258727, 12.1867, 29.2257] 	are coordinates for	t2_tse_tra_ProstateX_0042_s004.nii.gz	transformed to	(210, 210, 6)
[0.258727, 12.1867, 29.2257] 	are coordinates for	t2_tse_tra_ProstateX_0042_s009.nii.gz	transformed to	(210, 210, 6)
[8.92826, 59.9957, -1.69448] 	are coordinates for	t2_tse_tra_ProstateX_0043.nii.gz	transformed to	(163, 159, 16)
[-5.795073, 18.7907, 87.5082] 	are coordinates for	t2_tse_tra_ProstateX_0044.nii.gz	transformed to	(197, 204, 5)
[-4.47705, 12.5405, 17.9921] 	are coordinates for	t2_tse_tra_ProstateX_0045.nii.gz	transformed to	(216, 225, 12)
[-32.861, -2.20032, 46.9729] 	are coordinates for	t2_tse_tra_ProstateX_0046.nii.gz	transformed to	(139, 184, 11)
[-4.84385, 2.95186, 35.0858] 	are coordinates for	t2_tse_tra_ProstateX_0046.nii.gz	transformed to	(195, 171, 7)
[-24.2868, 62.7355, -24.2213] 	are coordinates for	t2_tse_tra_ProstateX_0047.nii.gz	tran

[29.4178, 43.3347, -62.3276] 	are coordinates for	t2_tse_tra_ProstateX_0093.nii.gz	transformed to	(193, 155, 14)
[9.6081, 31.5817, -68.8806] 	are coordinates for	t2_tse_tra_ProstateX_0093.nii.gz	transformed to	(154, 177, 11)
[30.4332, 39.6444, -98.2314] 	are coordinates for	t2_tse_tra_ProstateX_0093.nii.gz	transformed to	(199, 155, 2)
[31.5813, 15.6722, -42.8497] 	are coordinates for	t2_tse_tra_ProstateX_0094.nii.gz	transformed to	(189, 229, 8)
[-29.8824, 33.0609, 17.1315] 	are coordinates for	t2_tse_tra_ProstateX_0095_s004.nii.gz	transformed to	(170, 169, 10)
[-28.6965, 9.10563, 11.1672] 	are coordinates for	t2_tse_tra_ProstateX_0095_s006.nii.gz	transformed to	(173, 215, 7)
[14.1124, 22.6493, 22.4049] 	are coordinates for	t2_tse_tra_ProstateX_0096.nii.gz	transformed to	(162, 199, 7)
[-21.7875, 18.8555, 24.8761] 	are coordinates for	t2_tse_tra_ProstateX_0097_s009.nii.gz	transformed to	(170, 203, 11)
[-21.7875, 18.8555, 24.8761] 	are coordinates for	t2_tse_tra_ProstateX_0097_s005.nii.gz

[25.0155, 40.2724, -5.74252] 	are coordinates for	t2_tse_tra_ProstateX_0131.nii.gz	transformed to	(155, 176, 5)
[-2.44585, 41.3319, 33.2478] 	are coordinates for	t2_tse_tra_ProstateX_0132_s009.nii.gz	transformed to	(214, 170, 10)
[-2.44585, 41.3319, 33.2478] 	are coordinates for	t2_tse_tra_ProstateX_0132_s005.nii.gz	transformed to	(214, 170, 12)
[22.8645, 28.3757, -26.314] 	are coordinates for	t2_tse_tra_ProstateX_0133.nii.gz	transformed to	(195, 235, 10)
[24.5865, 41.3689, -11.8297] 	are coordinates for	t2_tse_tra_ProstateX_0134.nii.gz	transformed to	(218, 181, 12)
[10.4469, 54.7228, -17.7097] 	are coordinates for	t2_tse_tra_ProstateX_0134.nii.gz	transformed to	(189, 152, 11)
[-5.23868, 43.376, -40.263] 	are coordinates for	t2_tse_tra_ProstateX_0135.nii.gz	transformed to	(164, 155, 13)
[6.18072, 21.8572, -45.0361] 	are coordinates for	t2_tse_tra_ProstateX_0135.nii.gz	transformed to	(187, 194, 10)
[-8.32957, 19.2429, -49.2812] 	are coordinates for	t2_tse_tra_ProstateX_0136.nii.gz	trans

[7.0255, 21.6969, -29.8274] 	are coordinates for	t2_tse_tra_ProstateX_0168_s005.nii.gz	transformed to	(169, 180, 6)
[9.94036, 27.0209, 30.4277] 	are coordinates for	t2_tse_tra_ProstateX_0169.nii.gz	transformed to	(200, 176, 10)
[21.3805, 48.3445, 52.8735] 	are coordinates for	t2_tse_tra_ProstateX_0170.nii.gz	transformed to	(213, 172, 16)
[-0.565471, 47.1089, 41.5742] 	are coordinates for	t2_tse_tra_ProstateX_0170.nii.gz	transformed to	(169, 167, 12)
[0.425155639648438, 40.347412109375, 23.8209381103516] 	are coordinates for	t2_tse_tra_ProstateX_0171_s004.nii.gz	transformed to	(162, 158, 11)
[20.1323, 18.8425, 28.2502] 	are coordinates for	t2_tse_tra_ProstateX_0171_s006.nii.gz	transformed to	(201, 201, 10)
[3.961669921875, 10.2980422973633, 19.8878326416016] 	are coordinates for	t2_tse_tra_ProstateX_0172.nii.gz	transformed to	(186, 224, 2)
[-7.23110961914062, 43.0895080566406, 37.3951797485352] 	are coordinates for	t2_tse_tra_ProstateX_0172.nii.gz	transformed to	(164, 163, 9)
[-19.91405

In [94]:
t2_df.loc[t2_df.coordinates=='']

,ProxID,Name,fid,pos,WorldMatrix,ijk,TopLevel,SpacingBetweenSlices,VoxelSpacing,Dim,DCMSerDescr,DCMSerNum,match,coordinates
53,ProstateX-0031,t2_tse_tra0,1,8.08436 38.9556 13.8648,"0.5,1.02552e-010,6.91944e-017,-107.639,-9.4949...",231 209 17,NaN,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,5,t2_tse_tra_ProstateX_0031.nii.gz,
56,ProstateX-0031,t2_tse_tra0,2,-25.5225 39.8319 13.5071,"0.5,1.02552e-010,6.91944e-017,-107.639,-9.4949...",164 212 14,NaN,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,9,t2_tse_tra_ProstateX_0031.nii.gz,
91,ProstateX-0054,t2_tse_tra0,2,20.3595 31.1926 -6.71699,"0.5,1.02552e-010,-1.19538e-016,-98.0005,-9.988...",237 216 9,NaN,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,8,t2_tse_tra_ProstateX_0054.nii.gz,
94,ProstateX-0054,t2_tse_tra0,1,-2.10843 10.7156 -4.33744,"0.6,1.23062e-010,-1.19538e-016,-98.0505,-1.198...",160 146 8,NaN,3.0,"0.6,0.6,3",320x320x21x1,t2_tse_tra,13,t2_tse_tra_ProstateX_0054.nii.gz,
124,ProstateX-0074,t2_tse_tra0,1,2.58529 47.7414 -79.8186,"0.5,1.02552e-010,-4.64581e-017,-101.856,-1.019...",209 225 11,NaN,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0074.nii.gz,
157,ProstateX-0095,t2_tse_tra0,2,-28.6965 9.10563 11.1672,"0.5,1.02552e-010,7.33771e-017,-115.35,-1.01698...",173 168 8,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0095.nii.gz,
160,ProstateX-0095,t2_tse_tra0,1,-29.8824 33.0609 17.1315,"0.5,1.02552e-010,7.33771e-017,-115.35,-1.01698...",171 214 11,1.0,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,6,t2_tse_tra_ProstateX_0095.nii.gz,
166,ProstateX-0099,t2_tse_tra0,1,25.7022 40.664 65.1977,"0.5,1.02552e-010,-4.64581e-017,-89.5667,-1.019...",231 235 11,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,5,t2_tse_tra_ProstateX_0099.nii.gz,
169,ProstateX-0099,t2_tse_tra0,2,-16.1111 39.0076 65.3807,"0.5,1.02552e-010,-4.64581e-017,-89.5667,-1.019...",147 231 11,1.0,3.0,"0.5,0.5,3",384x384x21x1,t2_tse_tra,9,t2_tse_tra_ProstateX_0099.nii.gz,
173,ProstateX-0101,t2_tse_tra0,1,-24.515 7.71764 -28.1541,"0.499998,1.19379e-010,-0.00845051,-119.665,0.0...",190 172 3,NaN,3.0,"0.5,0.5,3",384x384x19x1,t2_tse_tra,4,t2_tse_tra_ProstateX_0101.nii.gz,


In [ ]:
a=[float(i) for i in (data_all['pos'].loc[data['ProxID']=='ProstateX-0000'][0]).split(' ')]

data_new['ProxID'][1]
data_new.loc[data_new['ProxID']=='ProstateX-0002'].index[0]

a

In [ ]:
data_all['pos'].loc[data['ProxID']=='ProstateX-0191']
directory

In [ ]:
for i in directory:
    for k in data_all['ProxID']:
        if k.split('-')[-1] in i: 
            print(k)
            pos=[float(i) for i in (data_all['pos'].loc[data_all['ProxID']==k][data_all.loc[data['ProxID']==k].index[0]]).split(' ')]
            print(pos)


In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([153-20, 166-32, 12])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')

In [ ]:
extract = sitk.ExtractImageFilter()
extract.SetSize([64,64,0])
extract.SetIndex([203-20, 168-32, 9])
extracted = extract.Execute(im)

extr_arr = sitk.GetArrayFromImage(extracted)
plt.imshow(extr_arr,cmap='gray')



In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/t2/t2_tse_tra_ProstateX_0002.nii')
im_arr = sitk.GetArrayFromImage(im)
im_sl_9 = im_arr[9,:,:]
im_sl_9 = sitk.GetImageFromArray(im_sl_9)

extract = sitk.ExtractImageFilter()
extract.SetSize([32,32])
extract.SetIndex([203-20, 168-32])
extracted_image = extract.Execute(im_sl_9)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
pos = [-27.0102, 41.5467, -26.0469]
ijk = im.TransformPhysicalPointToIndex(pos)

print(ijk)

In [ ]:
im = sitk.ReadImage('C://Image Processing/ProstateX/resampled/ep2d_diff_tra_DYNDIST_ADC_ProstateX_0000.nii')
im_arr = sitk.GetArrayFromImage(im)

im_sl_8 = im_arr[9,:,:]
im_sl_8 = sitk.GetImageFromArray(im_sl_8)

extract = sitk.ExtractImageFilter()
extract.SetSize([36,36])
extract.SetIndex([36, 72 ])
extracted_image = extract.Execute(im_sl_8)


extrcated_arr = sitk.GetArrayFromImage(extracted_image)

plt.imshow(extrcated_arr, cmap = 'gray')

In [ ]:
myshow(im_arr,9)

In [ ]:
data_all.loc[data_all['ProxID']=='ProstateX-0002']